# AI Agent Example 

## AI Agents
By themselves, large language models (LLMs) can't take actions — they just output text. Agents are systems that take a high-level task and use an LLM as a reasoning engine to decide what actions to take and execute those actions. After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish.

## AI Agent Example
In this example, an agent can have conversations with its users (humans or other agents). It can provide up-to-date information about the present that the LLM does not possess because it was not trained on it.

The agent:
 - Interacts with a search engine (an external tool) to answer users' questions. The engine is called [Tavily](https://www.tavily.com/) and provides a web search API that enables real-time information retrieval from the internet. It's useful for grounding LLMs with fresh, factual, and search-based answers.
 - Has conversational memory — meaning that it can be used as a multi-turn chatbot. The agent remembers past interactions across multiple turns of a conversation which in turn enables context-aware decision-making and more natural, coherent dialogue over time.

## Agentic Workflow
Here is an AI Agent sequence diagram:

 <center><img src="images/sequence_diagram.png"/></center>

 Source: [LangGraph](https://langchain-ai.github.io/langgraph/reference/agents/#langgraph.prebuilt.chat_agent_executor.create_react_agent)

 This example uses the [LangChain](https://python.langchain.com/) LLM framework, but there are others — such as OpenAI's tools or CrewAI — that can provide a similar experience.


## Framework Setup 

In [1]:
# LangChain
# https://python.langchain.com/docs/how_to/installation/
#%pip install -qU langgraph langchain-tavily langgraph-checkpoint-sqlite

In [2]:
#%pip install -qU python-dotenv

In [3]:
import os
import json

In [4]:
from dotenv import load_dotenv

load_dotenv(override=True)
MY_ENV_VAR = os.getenv('LANGSMITH_TRACING')
print(MY_ENV_VAR)

true


## Tools


For all available tools, see [LangChain providers](https://python.langchain.com/docs/integrations/providers/).

In [5]:
from langchain_tavily import TavilySearch

In [6]:
tools = []

In [7]:
search = TavilySearch(max_results=2)
search_results = search.invoke("What is the current USD to GBP exchange rate?")
print(json.dumps(search_results, sort_keys=True, indent=4))

# Add the tool to the reference list
tools.append(search)

{
    "answer": null,
    "follow_up_questions": null,
    "images": [],
    "query": "What is the current USD to GBP exchange rate?",
    "response_time": 0.93,
    "results": [
        {
            "content": "Convert US Dollar to British Pound ; 1 USD. 0.745395 GBP ; 5 USD. 3.72698 GBP ; 10 USD. 7.45395 GBP ; 25 USD. 18.6349 GBP ; 50 USD. 37.2698 GBP.",
            "raw_content": null,
            "score": 0.91399205,
            "title": "1 USD to GBP - US Dollars to British Pounds Exchange Rate - Xe",
            "url": "https://www.xe.com/currencyconverter/convert/?Amount=1&From=USD&To=GBP"
        },
        {
            "content": "Convert USD to GBP at the real exchange rate. Amount. USD. Converted to. GBP. $1.000 USD = \u00a30.7458 GBP. Mid-market exchange rate at 20:59. Track the exchange",
            "raw_content": null,
            "score": 0.9065293,
            "title": "USD to GBP - Convert US dollars to British pounds sterling - Wise",
            "url": "https://wi

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('400 Client Error: Bad Request for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Bad request: invalid \'dotted_order\': dotted_order 765b64c7-2541-44b0-bb69-2698c04a9fcf has timestamp 2025-07-19 21:13:25.108878 +0000 UTC earlier than parent timestamp 2025-07-19 21:14:31.022538 +0000 UTC for run_id:765b64c7-2541-44b0-bb69-2698c04a9fcf trace_id:9cace128-34ce-4b22-b198-f22ef66a7aa0 dotted_order:20250719T211431016399Z9cace128-34ce-4b22-b198-f22ef66a7aa0.20250719T211431022538Z598d37d7-058d-422f-b404-464951f90fcc.20250719T211325108878Z765b64c7-2541-44b0-bb69-2698c04a9fcf parent_run_id:598d37d7-058d-422f-b404-464951f90fcc"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('400 Client Error: B

In [8]:
tools

[TavilySearch(max_results=2, api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'), api_base_url=None))]

## Large Language Model

In [9]:
#%pip install -qU "langchain[openai]" # behind Open AI GPT, used by Azure OpenAI
# gpt-4o is now the default GPT-4 model as of mid-2025

%pip install -qU "langchain[anthropic]" # behind Claude
#%pip install -qU langchain-anthropic
#%pip install -qU "langchain[google-genai]" # behind Gemini
#%pip install -qU "langchain[google-vertexai]" # behind Vertex AI
#%pip install -qU "langchain[aws]" # behind Amazon Bedrock
#%pip install -qU "langchain[groq]" # behind Groq AI, used by Perplexity AI and Meta AI

Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain.chat_models import init_chat_model

#model = init_chat_model("gpt-4.1", model_provider="openai")
#model = init_chat_model("gpt-4o-mini", model_provider="openai") # default and cheapest GPT-4 model
model = init_chat_model("anthropic:claude-3-5-sonnet-latest")

In [11]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hello! How can I help you today?'

In [13]:
query = "What is the current USD to GBP exchange rate?"
response = model.invoke([{"role": "user", "content": query}])
response.text()

"I cannot provide real-time exchange rates as they fluctuate constantly throughout the day. To get the current USD to GBP exchange rate, I recommend:\n\n1. Checking financial websites like xe.com or Bloomberg\n2. Using your bank's online currency converter\n3. Looking up Google's currency converter\n4. Consulting a forex trading platform"

In [14]:
response.tool_calls

[]

In [15]:
# Bind the model to the tools
model_with_tools = model.bind_tools(tools)

In [16]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])
response.text()

"Hello! I'm here to help you search for information using the available search tool. However, I need to know what specific information you're looking for. Please let me know what topic or question you'd like to learn more about, and I'll be happy to help you find relevant and accurate information."

In [17]:
response.tool_calls

[]

In [18]:
#print(json.dumps(response.to_json(), sort_keys=True, indent=4))

In [19]:
query = "What is the current USD to GBP exchange rate?"
response = model_with_tools.invoke([{"role": "user", "content": query}])
response.text()

"I'll search for the current USD to GBP exchange rate using Tavily search."

In [20]:
response.tool_calls

[{'name': 'tavily_search',
  'args': {'query': 'current USD to GBP exchange rate', 'time_range': 'day'},
  'id': 'toolu_016RNV4mnUwWgteH8EEWnC1v',
  'type': 'tool_call'}]

## AI Agent Definition

In [21]:
#from langchain.chat_models import init_chat_model
#from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [22]:
memory = MemorySaver()
model = init_chat_model("anthropic:claude-3-5-sonnet-latest",
    temperature=0, # Lower temperature for more deterministic responses
    max_tokens=1000, # Response length limit, cost limitation, and performance optimization (ca. 75 tokens per 100 words)
    )
search = TavilySearch(max_results=2)
tools = [search]
agent_executor = create_react_agent(
    model, 
    tools, 
    checkpointer=memory, 
    prompt="You are a helpful assistant"
)

## Using AI Agent

In [23]:
config = {"configurable": {"thread_id": "a001"}}

input_message = {
    "role": "user",
    "content": "Hi, I'm Krzysztof and I live in London.",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Krzysztof and I live in London.
================================== Ai Message ==================================

Hello Krzysztof! Nice to meet you. I see you're based in London. While you've introduced yourself, you haven't asked any specific question or made any request that would require me to use the available tools. Is there something specific you'd like to know or learn about? I'd be happy to help you with any questions you might have!


In [24]:
input_message = {
    "role": "user",
    "content": "What's the weather where I live?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's the weather where I live?
================================== Ai Message ==================================

[{'text': 'Let me search for current weather information in London.', 'type': 'text'}, {'id': 'toolu_01Fq3cz6XhNCW9BLehLiV9e2', 'input': {'query': 'current weather London UK', 'time_range': 'day'}, 'name': 'tavily_search', 'type': 'tool_use'}]
Tool Calls:
  tavily_search (toolu_01Fq3cz6XhNCW9BLehLiV9e2)
 Call ID: toolu_01Fq3cz6XhNCW9BLehLiV9e2
  Args:
    query: current weather London UK
    time_range: day
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather London UK", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in London, UK", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'London', 'region': 'City of London, Greater London', 'country': 'United 

In [25]:
input_message = {
    "role": "user",
    "content": "What's the USD to GBP exchange rate?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's the USD to GBP exchange rate?
================================== Ai Message ==================================

[{'text': 'Let me search for the current USD to GBP exchange rate.', 'type': 'text'}, {'id': 'toolu_01W1VFeLWMZrwztuF6oqvvoy', 'input': {'query': 'current USD to GBP exchange rate', 'topic': 'finance', 'time_range': 'day'}, 'name': 'tavily_search', 'type': 'tool_use'}]
Tool Calls:
  tavily_search (toolu_01W1VFeLWMZrwztuF6oqvvoy)
 Call ID: toolu_01W1VFeLWMZrwztuF6oqvvoy
  Args:
    query: current USD to GBP exchange rate
    topic: finance
    time_range: day
================================= Tool Message =================================
Name: tavily_search

{"query": "current USD to GBP exchange rate", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.forbes.com/advisor/money-transfer/currency-converter/usd-gbp/", "title": "USD To GBP: Convert Un

In [26]:
input_message = {
    "role": "user",
    "content": "Where can I exchange money nearby?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Where can I exchange money nearby?
================================== Ai Message ==================================

[{'text': 'Let me search for currency exchange locations in London.', 'type': 'text'}, {'id': 'toolu_01EwoZCCPVsuZmbX7yjp3gQv', 'input': {'query': 'best currency exchange places London UK money exchange bureaus'}, 'name': 'tavily_search', 'type': 'tool_use'}]
Tool Calls:
  tavily_search (toolu_01EwoZCCPVsuZmbX7yjp3gQv)
 Call ID: toolu_01EwoZCCPVsuZmbX7yjp3gQv
  Args:
    query: best currency exchange places London UK money exchange bureaus
================================= Tool Message =================================
Name: tavily_search

{"query": "best currency exchange places London UK money exchange bureaus", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://wise.com/gb/blog/currency-exchange-london", "title": "Best places to exchange currency in L

In [27]:
for s in step["messages"]:
    s.pretty_print()

================================ Human Message =================================

Hi, I'm Krzysztof and I live in London.
================================== Ai Message ==================================

Hello Krzysztof! Nice to meet you. I see you're based in London. While you've introduced yourself, you haven't asked any specific question or made any request that would require me to use the available tools. Is there something specific you'd like to know or learn about? I'd be happy to help you with any questions you might have!
================================ Human Message =================================

What's the weather where I live?
================================== Ai Message ==================================

[{'text': 'Let me search for current weather information in London.', 'type': 'text'}, {'id': 'toolu_01Fq3cz6XhNCW9BLehLiV9e2', 'input': {'query': 'current weather London UK', 'time_range': 'day'}, 'name': 'tavily_search', 'type': 'tool_use'}]
Tool Calls:
  tavily_se